In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [10]:
load_dotenv(override=True)

True

In [16]:
def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key = os.getenv("SENDGRID_API_KEY"))
    from_email = Email("melodystitchqi@gmail.com")
    to_email = To("melodyq@workfx.ai")
    subject = "Good Morning"
    content = Content("text/plain", "and easy to do anywhere, even with Python")
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)
send_test_email()

202


In [12]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [13]:
model = "gpt-4o-mini"

In [14]:
sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model
)

sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model
)

sales_agent3= Agent(
    name="Consice Sales Agent",
    instructions=instructions3,
    model=model
)

In [18]:
result = Runner.run_streamed(sales_agent3, input="Write a code sales email")

async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Simplify SOC2 Compliance with AI

Hi [Recipient's Name],

I hope this message finds you well. I’m [Your Name] from ComplAI, and I wanted to reach out because we specialize in helping businesses like yours streamline SOC2 compliance using AI.

Our platform automates key compliance processes, saving you time and reducing audit preparation headaches. Imagine having all your compliance needs managed efficiently in one place.

Would you be open to a quick call next week to discuss how ComplAI can enhance your compliance strategy?

Looking forward to your response!

Best,  
[Your Name]  
[Your Contact Information]  
[ComplAI Website]  

In [19]:
sales_picker = Agent(
    name="Sales Picker",
    instructions = "You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model=model
)

In [20]:
message = "Write a cold sales email"
with trace("Selection from seles people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, input=message),
        Runner.run(sales_agent2, input=message),
        Runner.run(sales_agent3, input=message)
    )
    outputs = [result.final_output for result in results]
    emails = "Code sales email options:\n\n" +"\n\nEmail:\n\n".join(outputs)
    best = await Runner.run(sales_picker, emails)
    print("\n\nBest email:\n", best.final_output)



Best email:
 Subject: Let’s Make SOC2 Compliance as Easy as Ordering Pizza 🍕

Hi [First Name],

Ever feel like SOC2 compliance is more complicated than deciphering ancient hieroglyphics? 😩 You’re not alone! 

At ComplAI, we believe compliance shouldn't require a PhD in legalese or a night spent crying over spreadsheets. Our AI-powered tool is here to save the day—like a superhero, but with fewer capes and more compliance. 🦸‍♂️

Imagine breezing through audits and compliance checks while sipping your morning coffee. With ComplAI, you can:

- Automate the boring stuff (goodbye, manual riffraff!)
- Keep your team sane (no more late-night panic attacks over documents!)
- Impress your auditors (they might even give you a high-five!)

Let’s schedule a quick chat to see how we can turn your compliance chaos into a triumph. After all, who doesn’t want to be the hero of their own compliance story? 📖✨

Looking forward to your reply!

Best,  
[Your Name]  
[Your Job Title]  
ComplAI  
[Your Con